In [2]:
import boto3

In [3]:
import pandas as pd
import psycopg2
import json

In [4]:
import configparser
config=configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [5]:
KEY = config.get("AWS","KEY")
SECRET = config.get("AWS","SECRET")

In [6]:
DWH_CLUSTER_TYPE=config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES=config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE=config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER=config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB=config.get("DWH","DWH_DB")
DWH_DB_USER=config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD=config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT=config.get("DWH","DWH_PORT")
DWH_IAM_ROLE_NAME=config.get("DWH","DWH_IAM_ROLE_NAME")

(DWH_DB_USER,DWH_DB_PASSWORD,DWH_DB)

('awsuser', 'Password1', 'myfirstdb')

In [7]:
pd.DataFrame({"Param":["DWH_CLUSTER_TYPE","DWH_NUM_NODES","DWH_NODE_TYPE","DWH_CLUSTER_IDENTIFIER","DWH_DB","DWH_DB_USER",
"DWH_DB_PASSWORD","DWH_PORT","DWH_IAM_ROLE_NAME"],"Value":[DWH_CLUSTER_TYPE,DWH_NUM_NODES,DWH_NODE_TYPE,DWH_CLUSTER_IDENTIFIER,DWH_DB,
DWH_DB_USER,DWH_DB_PASSWORD,DWH_PORT,DWH_IAM_ROLE_NAME]})

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,my-first-redshift
4,DWH_DB,myfirstdb
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,Password1
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,redshift-s3-access


In [8]:
ec2=boto3.resource('ec2',region_name="ap-south-1",aws_access_key_id=KEY,aws_secret_access_key=SECRET)

In [9]:
s3 = boto3.resource('s3',region_name="ap-south-1",aws_access_key_id=KEY,aws_secret_access_key=SECRET)
iam = boto3.client('iam',region_name="ap-south-1",aws_access_key_id=KEY,aws_secret_access_key=SECRET)
redshift = boto3.client('redshift',region_name="ap-south-1",aws_access_key_id=KEY,aws_secret_access_key=SECRET)

In [10]:
bucket = s3.Bucket("ann-test-bucket1")
log_data_files= [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

[]

In [11]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [12]:
roleArn

'arn:aws:iam::313680870945:role/redshift-s3-access'

In [18]:
try :
    response = redshift.create_cluster(
    ClusterType=DWH_CLUSTER_TYPE,
    NodeType = DWH_NODE_TYPE,
        
    #Identifires & Credentials
    DBName = DWH_DB,
    ClusterIdentifier= DWH_CLUSTER_IDENTIFIER,
    MasterUsername = DWH_DB_USER,
    MasterUserPassword= DWH_DB_PASSWORD,
        
    #Roles ( for s3 access)
    IamRoles = [roleArn]
        
    )
except Exception as e :
    print(e)

In [21]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

{'Clusters': [{'ClusterIdentifier': 'my-first-redshift',
   'NodeType': 'dc2.large',
   'ClusterStatus': 'available',
   'ClusterAvailabilityStatus': 'Available',
   'MasterUsername': 'awsuser',
   'DBName': 'myfirstdb',
   'Endpoint': {'Address': 'my-first-redshift.ckhqduutsfs1.ap-south-1.redshift.amazonaws.com',
    'Port': 5439},
   'ClusterCreateTime': datetime.datetime(2022, 10, 20, 14, 26, 12, 750000, tzinfo=tzutc()),
   'AutomatedSnapshotRetentionPeriod': 1,
   'ManualSnapshotRetentionPeriod': -1,
   'ClusterSecurityGroups': [],
   'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0748fd3d5097d77bf',
     'Status': 'active'}],
   'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
     'ParameterApplyStatus': 'in-sync'}],
   'ClusterSubnetGroupName': 'default',
   'VpcId': 'vpc-04950c02689d0199e',
   'AvailabilityZone': 'ap-south-1a',
   'PreferredMaintenanceWindow': 'tue:06:00-tue:06:30',
   'PendingModifiedValues': {},
   'ClusterVersion': '1.0',
   'Allow

In [25]:
def PrettyRedshiftProps(props):
    pd.set_option('display.max_colwidth',-1)
    keysToShow=["ClusterIdentifier","NodeType","ClusterStatus","MasterUsername","DBName","Endpoint","VpcId"]
    x=[(k,v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x,columns=["Key","Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
PrettyRedshiftProps(myClusterProps)

C:\Users\ANERGH~1\AppData\Local\Temp/ipykernel_4620/3929763020.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


,Key,Value
0,ClusterIdentifier,my-first-redshift
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,myfirstdb
5,Endpoint,"{'Address': 'my-first-redshift.ckhqduutsfs1.ap-south-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-04950c02689d0199e


In [27]:
DWH_ENDPOINT= myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

In [28]:
DB_USER

'awsuser'